# Multiple simulations with a single progress bar

By default Nengo displays a progress bar for every simulation that takes longer than a certain time interval. When running a large number of simulations it would be more helpful to get a single progress bar that shows the progress of that outer loop. This example shows how this can be achieved.

## The nengo.ipynb extension

Within a Jupyter Notebook it is recommended to load the `nengo.ipynb` extension to get a better integration of the progress bar with the notebook. This is not required if you are running your code in the command line instead of a notebook.

In [1]:
%load_ext nengo.ipynb

<IPython.core.display.Javascript object>

## A simple Nengo model

To have something that we can run inside the loop, we define a simple Nengo model. Apart from that, it is not doing anything useful.

In [2]:
import nengo

with nengo.Network() as model:
    ens = nengo.Ensemble(500, 1)

## ProgressTracker

To display a progress bar, we use the ``ProgressTracker`` class which can be imported from `nengo.utils.progress`.

In [3]:
from nengo.utils.progress import ProgressTracker

To use this class, it has to be instantiated with three arguments: the total number of steps (the number of trials we want to run in this case), `True` (this argument allows further customization of the progress bar and isn't covered here), and a string describing the task. The instance is then used as part of a `with` statement that encloses the long running code for which the progress bar shall be displayed. Whenever we complete a step (trial), we call the `step` method on it to advance the progress.

Furthermore, we set the `progress_bar` argument of the `Simulator` to `False` to prevent an extra progress bar from popping up for each trial.

In [4]:
n_trials = 100
with ProgressTracker(n_trials, True, 'Running trials') as progress:
    for i in range(n_trials):
        with nengo.Simulator(model, progress_bar=False) as sim:
            sim.run(1.)
            progress.step()